In [ ]:
import time, threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException
from shutil import which
from openpyxl import load_workbook
from init_driver import init_driver
from count_dropdown_itens import count_dropdown_itens

wb = load_workbook('../BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

print("Coleta de CNPJs finalizada")

dados_diarios_list = []

def get_dados_diarios(index, driver, cnpj, stored_name, item_to_finish):

    driver.switch_to.default_content()

    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))

    print(f"Thread {index+1} iniciando coleta de dados diários")
    #dados_diarios_list = []
    
    driver.switch_to.default_content()

    WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))
    dropdown_data_pesquisa.select_by_index(item_to_finish)

    name_selected_dropdown = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))
    name_selected_dropdown = name_selected_dropdown.first_selected_option.get_attribute("value")
    
    linhas_tabela = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//table[@id='dgDocDiario']//tr[position()>1]"))
    )
    linhas_validas = [linha for linha in linhas_tabela if linha.find_element(By.XPATH, "td[2]").text.strip()] 
    
    if linhas_validas:
        ultima_linha = linhas_validas[-1]
        dados = ultima_linha.find_elements(By.XPATH, "td")
        dados_diarios = {
            "Cnpj": cnpj,
            "Nome": stored_name,
            "Mês": name_selected_dropdown,
            "Dia": dados[0].text.strip(),
            "Quota": dados[1].text.strip(),
            "Patrimônio Líquido": dados[4].text.strip(),
            "Número de Cotistas": dados[6].text.strip(),
        }
        dados_diarios_list.append(dados_diarios)
#dropdown.select_by_visible_text('02/2025')
    print(f"Thread {index+1} finalizou coleta de dados diários")
    if len(list_to_finish) == 0:
        print(f"Thread {index+1} finalizada")
        pass
    else:
        try:
            new_item_to_finish = list_to_finish.pop(0)
            print(list_to_finish)
        except IndexError:
            print(f"Thread {index+1} finalizada")
            pass
        print(f"Thread {index+1} reiniciando procura de dados diários")
        get_dados_diarios(index, driver, cnpj, stored_name, new_item_to_finish)

def get_info(cnpj, index, item_to_finish):
    try:
        print(f"Thread {index+1} iniciada para o CNPJ {cnpj}")
        driver = init_driver(index)
        print(f"Driver da Thread {index+1} iniciado com sucesso")

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
        )

        search_input.clear()
        #search_input.send_keys(lista_cnpj[0])
        search_input.send_keys(cnpj)
        search_input.send_keys(Keys.ENTER)
        print(f"Thread {index+1} pesquisou o CNPJ {cnpj}")

        try:
            find_name_to_store = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
            )
        except TimeoutException:
            find_name_to_store = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
            )

        stored_name = find_name_to_store.text
        find_name_to_store.click()

        driver.switch_to.default_content()

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        dados_diarios = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
        )
        dados_diarios.click()
        print(f"Thread {index+1} clicou em dados diários")

        driver.switch_to.default_content()

        #pop-up com mensagem irrelevante aparece e precisa ser fechado.
        try:
            alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
            alert.accept()
        except:
            pass

        get_dados_diarios(index, driver, cnpj, stored_name, item_to_finish)

        
    except TimeoutException:
        print(f"Erro na Thread {index+1}")
        driver.quit()
        print(f"Thread {index+1} reiniciando...")
        get_info(cnpj, index, item_to_finish)
    driver.quit()

list_cnpj_teste = ["58.878.941/0001-02", "36.248.874/0001-00", "51.017.442/0001-81", "57.270.020/0001-08"]

threads = []

# Criando threads em blocos de 8
for cnpj in list_cnpj_teste:
    print("Coleta tamanho do dropdown iniciada")
    tamanho = count_dropdown_itens(cnpj)
    print("Tamanho do dropdown coletado com sucesso")
    list_to_finish = list(range(tamanho))
    for index in range(6):
        try:
            thread = threading.Thread(target=get_info, args=(cnpj, index, list_to_finish.pop(0)))
            print(list_to_finish)
            threads.append(thread)
            thread.start()
        except IndexError:
            pass

# Aguardando todas as threads terminarem
for thread in threads:
    thread.join()

print("Todas as Threads foram finalizadas")

Coleta de CNPJs finalizada
Coleta tamanho do dropdown iniciada
Tamanho do dropdown coletado com sucesso
[1]
Thread 1 iniciada para o CNPJ 58.878.941/0001-02
[]
Thread 2 iniciada para o CNPJ 58.878.941/0001-02
Coleta tamanho do dropdown iniciada
Driver da Thread 2 iniciado com sucesso
Thread 2 pesquisou o CNPJ 58.878.941/0001-02
Driver da Thread 1 iniciado com sucesso
Thread 1 pesquisou o CNPJ 58.878.941/0001-02
Thread 2 clicou em dados diários
Thread 1 clicou em dados diários
Tamanho do dropdown coletado com sucesso
Thread 2 iniciando coleta de dados diários
Thread 1 iniciando coleta de dados diários
Thread 2 finalizou coleta de dados diários
Thread 2 finalizada
Thread 1 finalizou coleta de dados diários
Thread 1 finalizada
[1, 2, 3]
Thread 1 iniciada para o CNPJ 36.248.874/0001-00
[2, 3]
Thread 2 iniciada para o CNPJ 36.248.874/0001-00
[3]
Thread 3 iniciada para o CNPJ 36.248.874/0001-00
[]
Thread 4 iniciada para o CNPJ 36.248.874/0001-00
Driver da Thread 4 iniciado com sucesso
Thread

In [17]:
import pandas as pd
testing = pd.DataFrame(dados_diarios_list)
testing['Mês'] = pd.to_datetime(testing['Mês'], format='%m/%Y')
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)

testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-10-01,31,"1.195,32948236","13.024.996,62",6
1,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,2024-10-01,31,"1,00364209","111.210.122,22",1
2,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-11-01,29,"1,5316007","368.876.093,12",56
3,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-11-01,29,"1.204,61210793","13.126.145,44",6
4,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,2024-11-01,29,"1,01992736","113.014.637,24",1
5,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-12-01,31,"1,5486839","372.740.265,23",54
6,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-12-01,31,"1.253,2158189","13.935.343,64",6
7,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,2024-12-01,31,"1,04432669","115.718.242,69",1
8,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2025-01-01,31,"1,566572","373.149.451,52",55
9,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2025-01-01,31,"1.277,94733581","14.985.526,18",6


In [13]:
testing_sorted = testing.sort_values(by=['Cnpj', 'Mês'], ignore_index=True)
testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-11-01,29,"1,5316007","368.876.093,12",56
1,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-12-01,31,"1,5486839","372.740.265,23",54
2,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2025-01-01,31,"1,566572","373.149.451,52",55
3,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2025-02-01,17,"1,5767656","375.594.588,28",55
4,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-10-01,31,"1.195,32948236","13.024.996,62",6
5,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-11-01,29,"1.204,61210793","13.126.145,44",6
6,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-12-01,31,"1.253,2158189","13.935.343,64",6
7,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2025-01-01,31,"1.277,94733581","14.985.526,18",6
8,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2025-02-01,17,"1.288,43244238","15.108.477,13",6
9,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,2024-10-01,31,"1,00364209","111.210.122,22",1


In [60]:
testing_sorted.to_excel('teste.xlsx', index=False)

In [38]:
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)
testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-04-01,29,"1,000378","1.156.043,65",6
1,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-05-01,31,"1,009178","43.118.555,31",41
2,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-06-01,30,"1,018104","51.178.462,87",68
3,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-07-01,29,"1,027238","86.750.252,95",93
4,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-08-01,31,"1,037549","127.852.265,60",111
...,...,...,...,...,...,...,...
162,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-10-01,31,"2,939227238","4.036.357.184,84",1030
163,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-11-01,29,"2,960462845","4.217.186.353,15",1045
164,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-12-01,31,"2,984656712","4.174.516.271,82",1035
165,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2025-01-01,31,"3,012876808","4.468.412.591,20",1058


In [5]:
tamanho = 6
to_int = int(tamanho)

In [8]:
a = list(range(to_int))

In [19]:
b = a.pop(0)
print(b)

IndexError: pop from empty list

In [1]:
import psutil

def kill_webdrivers():
    for process in psutil.process_iter(attrs=['pid', 'name']):
        if process.info['name'] and ('chromedriver' in process.info['name'].lower() or 'geckodriver' in process.info['name'].lower()):
            print(f"Matando processo {process.info['name']} (PID: {process.info['pid']})")
            psutil.Process(process.info['pid']).terminate()  # Encerra o processo

kill_webdrivers()


Matando processo chromedriver.exe (PID: 3344)
Matando processo chromedriver.exe (PID: 4468)
Matando processo chromedriver.exe (PID: 13032)
Matando processo chromedriver.exe (PID: 16084)
Matando processo chromedriver.exe (PID: 26028)
Matando processo chromedriver.exe (PID: 29024)
Matando processo chromedriver.exe (PID: 30716)
Matando processo chromedriver.exe (PID: 32208)
Matando processo chromedriver.exe (PID: 32624)
Matando processo chromedriver.exe (PID: 33052)
Matando processo chromedriver.exe (PID: 33964)
Matando processo chromedriver.exe (PID: 34540)
Matando processo chromedriver.exe (PID: 35452)
Matando processo chromedriver.exe (PID: 36744)
